## Import necessary libraries

In [29]:
%reset -f

import pickle
import pandas as pd
import numpy
import time
from sklearn.gaussian_process import GaussianProcessClassifier
from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import fbeta_score
from sklearn.metrics import make_scorer

## Save environment parameters, so those won't be saved to report file

In [30]:
environment_variables = dir()

## Define model related constants

In [31]:
# Input parameters
INPUT_TRAINING_FILE = "../data/original/training_dataset.csv"
INPUT_EVALUATION_FILE = "../data/original/evaluation_dataset.csv"

# Output parameters
METHOD_NAME = "gaussian_process"
TIMESTAMP = time.strftime("%d_%m_%Y-%H_%M_%S")
OUTPUT_MODEL = f"../data/models/{METHOD_NAME}_model_{TIMESTAMP}.pkl"
OUTPUT_RESULTS = f"../data/results/{METHOD_NAME}_model_{TIMESTAMP}.txt"

# Hyper parameter alternatives
HYPER_PARAMETER_K_FEATURES = list(range(5, 30, 5))
HYPER_PARAMETER_SCORE_FUNC = [chi2]
# HYPER_PARAMETER_N_COMPONENTS = [3, 5, 7, 10, 15]
HYPER_PARAMETER_N_RESTARTS_OPTIMIZER = list(range(0, 2, 1))

# Hyper parameter optimization parameters
# HYPER_PARAMETER_OPTIMIZATION_SCORING = "balanced_accuracy"
HYPER_PARAMETER_OPTIMIZATION_BETA = 2
HYPER_PARAMETER_OPTIMIZATION_CV = 5

# Other constants
LABELS = ["Operational", "Bankrupt"]
RANDOM_SEED = 42
VERBOSITY = 3

## Load the datasets
These dataset should contain "Bankrupt?" label and features that should be considered during learning.

In [32]:
training_dataset = pd.read_csv(INPUT_TRAINING_FILE, engine="python", delimiter=",")
training_features = training_dataset.loc[:, training_dataset.columns != "Bankrupt?"]
training_targets = training_dataset["Bankrupt?"]

evaluation_dataset = pd.read_csv(INPUT_EVALUATION_FILE, engine="python", delimiter=",")
evaluation_features = evaluation_dataset.loc[:, evaluation_dataset.columns != "Bankrupt?"]
evaluation_targets = evaluation_dataset["Bankrupt?"]

## Minor preprocessing
Remove constant Net Income Flag and  Liability-Assets Flag features.

In [33]:
training_features = training_features.drop(columns=[" Net Income Flag"])
evaluation_features = evaluation_features.drop(columns=[" Net Income Flag"])
training_features = training_features.drop(columns=[" Liability-Assets Flag"])
evaluation_features = evaluation_features.drop(columns=[" Liability-Assets Flag"])

## Define the training pipeline

In [34]:
# Create internal pipeline
classifier = GaussianProcessClassifier()
standard_scaler =  StandardScaler()
min_max_scaler =  MinMaxScaler()
feature_selection = SelectKBest()
# pca = PCA(random_state=RANDOM_SEED)
smote = SMOTE(sampling_strategy="minority", random_state=RANDOM_SEED)
pipeline = Pipeline(steps=[("standard_scaler", standard_scaler),("min_max_scaler", min_max_scaler),
    ("selection", feature_selection), ("smote", smote), ("classification", classifier)])

# Specify the tunable hyper parameters
parameters = {
    "selection__k": HYPER_PARAMETER_K_FEATURES,
    "selection__score_func": HYPER_PARAMETER_SCORE_FUNC,
    "classification__n_restarts_optimizer": HYPER_PARAMETER_N_RESTARTS_OPTIMIZER
}

# Define KFold parameters
cv = StratifiedKFold(n_splits=HYPER_PARAMETER_OPTIMIZATION_CV, shuffle=True, random_state=RANDOM_SEED)

# Define custom fbeta scorer function that put emphasis on recall
def custom_scorer(y_true, y_pred):
    return fbeta_score(y_true, y_pred, beta=HYPER_PARAMETER_OPTIMIZATION_BETA)

estimator = GridSearchCV(pipeline, parameters, verbose=VERBOSITY,
    scoring=make_scorer(custom_scorer), cv=cv, n_jobs=3)

## Do training

In [35]:
estimator.fit(training_features, training_targets)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END classification__n_restarts_optimizer=0, selection__k=5, selection__score_func=<function chi2 at 0x7fae66718040>;, score=0.153 total time= 3.5min
[CV 2/5] END classification__n_restarts_optimizer=0, selection__k=5, selection__score_func=<function chi2 at 0x7fc41afd6040>;, score=0.158 total time= 3.5min
[CV 3/5] END classification__n_restarts_optimizer=0, selection__k=5, selection__score_func=<function chi2 at 0x7f55df0f8040>;, score=0.209 total time= 3.9min
[CV 4/5] END classification__n_restarts_optimizer=0, selection__k=5, selection__score_func=<function chi2 at 0x7fae66718040>;, score=0.234 total time= 3.6min
[CV 5/5] END classification__n_restarts_optimizer=0, selection__k=5, selection__score_func=<function chi2 at 0x7fc41afd6040>;, score=0.404 total time= 3.7min
[CV 1/5] END classification__n_restarts_optimizer=0, selection__k=5, selection__score_func=<function f_classif at 0x7f55df1a6ee0>;, score=0.453 tota

## Calculate metric values

In [ ]:
def confusion_matrix_to_string(cm):
    return f"True operational={cm[0][0]}, True bankrupt={cm[1][1]}, False operational={cm[0][1]}, False bankrupt={cm[1][0]}"

# Use all training data to calculate confusion matrix for training data
training_estimates = estimator.predict(training_features)
training_accuracy = balanced_accuracy_score(training_targets, training_estimates)
training_confusion_matrix = confusion_matrix(training_targets, training_estimates)
training_confusion_matrix = confusion_matrix_to_string(training_confusion_matrix)
training_classification_report = classification_report(training_targets, training_estimates, output_dict=True, target_names=LABELS)
training_f_beta_score = fbeta_score(training_targets, training_estimates, beta=HYPER_PARAMETER_OPTIMIZATION_BETA)

# Use model to estimate manually labeled evaluation Tweets
evaluation_estimates = estimator.predict(evaluation_features)
evaluation_accuracy = balanced_accuracy_score(evaluation_targets, evaluation_estimates)
evaluation_confusion_matrix = confusion_matrix(evaluation_targets, evaluation_estimates)
evaluation_confusion_matrix = confusion_matrix_to_string(evaluation_confusion_matrix)
evaluation_classification_report = classification_report(evaluation_targets, evaluation_estimates, output_dict=True, target_names=LABELS)
evaluation_f_beta_score = fbeta_score(evaluation_targets, evaluation_estimates, beta=HYPER_PARAMETER_OPTIMIZATION_BETA)

# Store best parameters
best_parameters = estimator.best_params_
print(best_parameters)

## Save trained model for future reference

In [ ]:
with open(OUTPUT_MODEL, "wb") as handle:
    pickle.dump(estimator, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Save metadata and calculated statistics
This is done by saving every textual and numerical variable to file.

In [ ]:
current_variables = dir()

results_string = ""

for variable in current_variables:
    # Skip environment variables and their container variable
    # Ignore also underscore variables
    if variable in environment_variables or variable == "environment_variables" or variable.startswith("_"):
        continue

    # Get variables value
    variable_value = globals()[variable]

    # If variable is numerical or string, append it to results
    if type(variable_value) is str or type(variable_value) is int or \
        type(variable_value) is float or type(variable_value) is list or \
        type(variable_value) is numpy.float64 or type(variable_value) is dict:
        results_string += f"{variable}: {variable_value}\n"

# Print results to screen
print(results_string)

# Save results to file
with open(OUTPUT_RESULTS, "w") as file:
    file.write(results_string)